벡터가 좋은 예 -> sympy로 행렬 연산 표현 -> 벡터 연산 코드 작성 -> 연산 결과를 sympy로 보이기

행렬 벡터는 복합 데이터Compound data를 표현하는 좋은 예입니다.

여기서는 이전에 만들었던 [accumulate](http://wikibootup.github.io/sicp/6-accumulate.html)를 이용하여 벡터의 기본 연산을 구현하도록 하겠습니다.

In [1]:
from modules.basic import accumulate, accumulate_n

먼저, 내적을 구해보겠습니다.

두 벡터의 1. 각 성분을 곱한 결과를 2. 모두 더하면 그게 내적이므로 ( $\sum_iv_iw_i$ )

1.의 과정은 `map`을 이용해서,

2.의 과정은 `accumulate`를 이용하여 코드를 구현하면,

In [2]:
def dot_product(v, w):
    return accumulate(
        lambda p1, p2: p1 + p2,
        0,
        list(map(lambda m1, m2: m1 * m2, v, w)))

아래의 한 문제만 검증해보면,

$
    A =
    \begin{pmatrix}
    1\ 2\ 3\ 4
    \end{pmatrix},
    \ B =
    \begin{pmatrix}
    5\ 6\ 7\ 8
    \end{pmatrix}
    ,\ A·B = B·A =\ ?
$

In [3]:
A = [1, 2, 3, 4]
B = [5, 6, 7, 8]

In [4]:
dot_product(A, B) == dot_product(B, A)

True

In [5]:
dot_product(A, B)

70